In [54]:
import yaml
import yaml
from yaml.loader import SafeLoader
import sys
sys.path.append("..")
import src.evaluation.eraserbenchmark.rationale_benchmark.utils as EU
import src.evaluation.eraserbenchmark.rationale_benchmark.metrics as EM
from src.data.locations import LOC

# load predictions and attentions
with open('../data/experiments/default/predictions_attentions.yaml') as f:
    data = yaml.load(f, Loader=SafeLoader)

# get documents
docids = list(data.keys())
documents = EM.load_flattened_documents('../'+LOC['cose'], docids=docids)

In [55]:
# all keys are the same (len will then also be the same)
list(data.keys()) == docids

True

In [56]:
# len of attention vectors == len of document tokens
[len(x['attn']) for x in list(data.values())] == [len(x) for x in list(documents.values())]

True

In [59]:
i = 2
list(zip(documents[docids[i]], data[docids[i]]['attn']))

[('Where', 0.7210550393651709),
 ('can', 0.8136533739680836),
 ('a', 0.0),
 ('human', 0.06314977894641013),
 ('find', 0.3412154787371332),
 ('clothes', 0.9999999999999998),
 ('that', 0.5786600416338707),
 ('are', 0.39342262859525895),
 ("n't", 0.2664831932688572),
 ('pants', 0.7014948502859857),
 ('?', 0.2664831932688572)]